In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
#import os
#import time
import requests
#import warnings
#warnings.filterwarnings('ignore')

In [2]:
with Browser('chrome') as browser:
    # Visit URL
    url_nasa = 'https://mars.nasa.gov/news/'
    browser.visit(url_nasa)
    #checking if the word Mars is present anywhere on the homepage to verify we found the website.
    if browser.is_text_present('Mars'):
        print("Yes, the official website was found!")
    else:
        print("No, it wasn't found... We need to improve our SEO techniques")


    #creating a BEautifulSoup object
    soup = bs(browser.html, 'html.parser')
    #reading the latest News Title
    news_title = soup.find_all('div', class_='content_title')[0].text
    #reading the latest news body
    paragraph = soup.find_all('div', class_="article_teaser_body")[0].text
    #printing the title and body just to see them and check that they are correct
    print(f'{news_title}\n{paragraph}')
    
    browser.quit()

Yes, the official website was found!
Mars Now
Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  


In [18]:
with Browser('chrome') as browser:
    # Visit URL
    url_jpl = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
    browser.visit(url_jpl)
    #checking if the word Mars is present anywhere on the homepage to verify we found the website.
    if browser.is_text_present('Mars'):
        print("Yes, the official website was found!")
    else:
        print("No, it wasn't found... We need to improve our SEO techniques")
    #clicking on the button FULL IMAGE to see the full image featured this month  class="btn btn-outline-light"
    browser.find_by_css('.btn.btn-outline-light').click()
    


Yes, the official website was found!
<html class="fancybox-margin fancybox-lock"><head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
      <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css">
      <!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
      <link rel="stylesheet" type="text/css" href="css/app.css">
      <link rel="stylesheet" type="text/css" href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css">
      <title>Space Image</title>
   <style type="text/css">.fancybox-margin{margin-right:17px;}</style></head>
   <body>
      <div class="header">
         <nav class="navbar navbar-expand-lg ">
            <a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"><span class="logo">Jet Propulsion Laboratory</span>
            <span class="logo1">California Institute of Technology</span></a>
            <button cl